# 시맨틱 커널 도구 사용 예제

이 문서는 ChromaDB와 통합된 Retrieval-Augmented Generation (RAG)을 구현하는 시맨틱 커널 기반 도구를 만드는 데 사용된 코드의 개요와 설명을 제공합니다. 이 예제는 ChromaDB 컬렉션에서 여행 문서를 검색하고, 사용자 쿼리를 시맨틱 검색 결과로 보완하며, 상세한 여행 추천을 스트리밍하는 AI 에이전트를 구축하는 방법을 보여줍니다.


SQLite 버전 수정  
다음 오류가 발생할 경우:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

노트북 시작 부분에서 이 코드 블록의 주석을 해제하세요:  


In [ ]:
# %pip install pysqlite3-binary
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### 패키지 가져오기
다음 코드는 필요한 패키지를 가져옵니다:


In [ ]:
import json
import os
import chromadb
from typing import Annotated, TYPE_CHECKING

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

if TYPE_CHECKING:
    from chromadb.api.models.Collection import Collection

### 시맨틱 커널 및 AI 서비스 생성

시맨틱 커널 인스턴스는 비동기 OpenAI 채팅 완료 서비스를 사용하여 생성되고 구성됩니다. 이 서비스는 응답 생성을 위해 커널에 추가됩니다.


In [ ]:
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)


# Create the OpenAI Chat Completion Service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

### 프롬프트 플러그인 정의하기

PromptPlugin은 검색 컨텍스트를 사용하여 확장된 프롬프트를 생성하는 함수를 정의하는 기본 플러그인입니다.


In [ ]:
class PromptPlugin:

    def __init__(self, collection: "Collection"):
        self.collection = collection

    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context."
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide your answer."
        )
    
    @kernel_function(name="retrieve_context", description="Retrieve context from the database.")
    def get_retrieval_context(self, query: str) -> str:
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_entries = []
        if results and results.get("documents") and results["documents"][0]:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_entries.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_entries) if context_entries else "No retrieval context found."

### 날씨 정보 플러그인 정의

WeatherInfoPlugin은 특정 여행지의 온도 정보를 제공하는 네이티브 플러그인입니다.


In [ ]:
class WeatherInfoPlugin:
    """A Plugin that provides the average temperature for a travel destination."""

    def __init__(self):
        # Dictionary of destinations and their average temperatures
        self.destination_temperatures = {
            "maldives": "82°F (28°C)",
            "swiss alps": "45°F (7°C)",
            "african safaris": "75°F (24°C)"
        }

    @kernel_function(description="Get the average temperature for a specific travel destination.")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "Returns the average temperature for the destination."]:
        """Get the average temperature for a travel destination."""
        # Normalize the input destination (lowercase)
        normalized_destination = destination.lower()

        # Look up the temperature for the destination
        if normalized_destination in self.destination_temperatures:
            return f"The average temperature in {destination} is {self.destination_temperatures[normalized_destination]}."
        else:
            return f"Sorry, I don't have temperature information for {destination}. Available destinations are: Maldives, Swiss Alps, and African safaris."

### 목적지 정보 플러그인 정의

DestinationsPlugin은 인기 있는 여행지에 대한 상세 정보를 제공하는 네이티브 플러그인입니다.


In [ ]:
class DestinationsPlugin:
    # Destination data store with rich details about popular travel locations
    DESTINATIONS = {
        "maldives": {
            "name": "The Maldives",
            "description": "An archipelago of 26 atolls in the Indian Ocean, known for pristine beaches and overwater bungalows.",
            "best_time": "November to April (dry season)",
            "activities": ["Snorkeling", "Diving", "Island hopping", "Spa retreats", "Underwater dining"],
            "avg_cost": "$400-1200 per night for luxury resorts"
        },
        "swiss alps": {
            "name": "The Swiss Alps",
            "description": "Mountain range spanning across Switzerland with picturesque villages and world-class ski resorts.",
            "best_time": "December to March for skiing, June to September for hiking",
            "activities": ["Skiing", "Snowboarding", "Hiking", "Mountain biking", "Paragliding"],
            "avg_cost": "$250-500 per night for alpine accommodations"
        },
        "safari": {
            "name": "African Safari",
            "description": "Wildlife viewing experiences across various African countries including Kenya, Tanzania, and South Africa.",
            "best_time": "June to October (dry season) for optimal wildlife viewing",
            "activities": ["Game drives", "Walking safaris", "Hot air balloon rides", "Cultural village visits"],
            "avg_cost": "$400-800 per person per day for luxury safari packages"
        },
        "bali": {
            "name": "Bali, Indonesia",
            "description": "Island paradise known for lush rice terraces, beautiful temples, and vibrant culture.",
            "best_time": "April to October (dry season)",
            "activities": ["Surfing", "Temple visits", "Rice terrace trekking", "Yoga retreats", "Beach relaxation"],
            "avg_cost": "$100-500 per night depending on accommodation type"
        },
        "santorini": {
            "name": "Santorini, Greece",
            "description": "Stunning volcanic island with white-washed buildings and blue domes overlooking the Aegean Sea.",
            "best_time": "Late April to early November",
            "activities": ["Sunset watching in Oia", "Wine tasting", "Boat tours", "Beach hopping", "Ancient ruins exploration"],
            "avg_cost": "$200-600 per night for caldera view accommodations"
        }
    }

    @kernel_function(
        name="get_destination_info",
        description="Provides detailed information about specific travel destinations."
    )
    def get_destination_info(self, query: str) -> str:
        # Find which destination is being asked about
        query_lower = query.lower()
        matching_destinations = []

        for key, details in DestinationsPlugin.DESTINATIONS.items():
            if key in query_lower or details["name"].lower() in query_lower:
                matching_destinations.append(details)

        if not matching_destinations:
            return (f"User Query: {query}\n\n"
                    f"I couldn't find specific destination information in our database. "
                    f"Please use the general retrieval system for this query.")

        # Format destination information
        destination_info = "\n\n".join([
            f"Destination: {dest['name']}\n"
            f"Description: {dest['description']}\n"
            f"Best time to visit: {dest['best_time']}\n"
            f"Popular activities: {', '.join(dest['activities'])}\n"
            f"Average cost: {dest['avg_cost']}" for dest in matching_destinations
        ])

        return (f"Destination Information:\n{destination_info}\n\n"
                f"User Query: {query}\n\n"
                "Based on the above destination details, provide a helpful response "
                "that addresses the user's query about this location.")

## ChromaDB 설정하기

Retrieval-Augmented Generation을 지원하기 위해, 지속적인 ChromaDB 클라이언트를 생성하고 `"travel_documents"`라는 이름의 컬렉션을 만듭니다(이미 존재하는 경우 가져옵니다). 그런 다음 이 컬렉션에 샘플 여행 문서와 메타데이터를 추가합니다.


In [ ]:
collection = chromadb.PersistentClient(path="./chroma_db").create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True,
)

documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours.",
]

collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[DestinationsPlugin(), WeatherInfoPlugin(), PromptPlugin(collection)],
    name="TravelAgent",
    instructions="Answer travel queries using the provided context. If context is provided, do not say 'I have no context for that.'",
)

### 스트리밍 채팅 기록과 함께 에이전트 실행하기
주요 비동기 루프는 대화의 채팅 기록을 생성하며, 각 사용자 입력에 대해 먼저 증강된 프롬프트(시스템 메시지로서)를 채팅 기록에 추가하여 에이전트가 검색 컨텍스트를 볼 수 있도록 합니다. 사용자 메시지도 추가된 후, 스트리밍을 사용하여 에이전트를 호출합니다. 출력은 스트리밍되는 대로 출력됩니다.


In [ ]:
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?",
        "What is the average temperature of the Maldives?",
        "What is a good cold destination offered by Contoso and what is it average temperature?",
    ]

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
